# prompting technique 

Take a latest news from newspaper which LLM will not have answer to e.g. 
on 27 Dec 2025 - Taiwan is rattled by earthquake of magniture 7.0

In [1]:
from dotenv import load_dotenv 
load_dotenv(override=True)

True

In [2]:
import os
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

# create llms 
google_llm = ChatGoogleGenerativeAI(temperature=0, model="gemini-3-flash-preview")
openai_llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

In [42]:
# Lets give some context to the LLM for question which will be asked later
context = """ 
A massive earthquake of magnitude 7.0 on the Richter Scale struck Taiwan on Saturday 27 Dec 2025, 
according to the country's weather agency, reported news agency AFP.
"""

In [43]:
# create system and human prompts 
# from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# create system prompt with and without context 
system_prompt_without_context = """ 
You are helpful assistant that helps people find information. You are given a question and you need to provide detailed and accurate answer.
Answer with "I don't know" if you are not absolutely sure.
"""

system_prompt_with_context = """ 
You are helpful assistant that helps people find information. You are given a question and you need to provide detailed and accurate answer.
Use context to answer: 
{context}

Answer with I don't know if you are not absolutely sure. 

"""

In [44]:
user_prompt = """ What was the magnitude of the earthquake recorded in Taiwan on 27 Dec 2025?"""

In [46]:
# lets create the prompt for llm 
from langchain_core.prompts import ChatPromptTemplate

prompt_template_without_context = ChatPromptTemplate.from_messages([
    ("system", system_prompt_without_context),
    ("user", "{user_prompt}")
])


prompt_template_with_context = ChatPromptTemplate.from_messages([
    ("system", system_prompt_with_context),
    ("user", "{user_prompt}")
])

In [47]:
prompt_template_without_context.input_variables

['user_prompt']

In [48]:
prompt_template_with_context.input_variables

['context', 'user_prompt']

In [49]:
prompt_template_without_context, \
prompt_template_with_context

(ChatPromptTemplate(input_variables=['user_prompt'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template=' \nYou are helpful assistant that helps people find information. You are given a question and you need to provide detailed and accurate answer.\nAnswer with "I don\'t know" if you are not absolutely sure.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['user_prompt'], input_types={}, partial_variables={}, template='{user_prompt}'), additional_kwargs={})]),
 ChatPromptTemplate(input_variables=['context', 'user_prompt'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template=" \nYou are helpful assistant that helps people find information. You are given a question and you need to provide detailed and accurate a

In [50]:
# First call without context to see if LLM knows 
pipeline_without_context = (
    {"user_prompt": lambda x : x["user_prompt"]}
    | prompt_template_without_context
    | google_llm
)

answer = pipeline_without_context.invoke({"user_prompt" : user_prompt})
answer.content[0]['text']

"I don't know."

In [51]:
# First call without context to see if LLM knows 
pipeline_with_context = (
    {
        "user_prompt": lambda x : x["user_prompt"], 
        "context" : lambda x : x["context"]
    }
    | prompt_template_with_context
    | google_llm
)

answer = pipeline_with_context.invoke({"user_prompt" : user_prompt, "context": context})
answer.content[0]['text']

'The magnitude of the earthquake recorded in Taiwan on 27 Dec 2025 was 7.0 on the Richter Scale.'

# Few shot prompting 
Not needed in Frontier model now. However for local LLM we can provide some of the examples, using these examples LLM has to respond in that way.

In [59]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"), 
    ("ai", "{output}")
])

In [60]:
examples = [
    {"input": "here is query #1", "output": "# Here is answer for #1"},
    {"input": "here is query #2", "output": "# Here is answer for #2"},
    {"input": "here is query #3", "output": "# Here is answer for #3"}
]

In [61]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

fewshot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

In [62]:
print(fewshot_prompt.format())

Human: here is query #1
AI: # Here is answer for #1
Human: here is query #2
AI: # Here is answer for #2
Human: here is query #3
AI: # Here is answer for #3


In [63]:
new_system_prompt = """
You are helpful assistant that helps people find information. You are given a question and you need to provide detailed and accurate answer.
Always answer in markdown format
Use context to answer: 
{context}

Answer with I don't know if you are not absolutely sure. 
"""

In [64]:
prompt_template_with_context.messages[0].prompt.template

" \nYou are helpful assistant that helps people find information. You are given a question and you need to provide detailed and accurate answer.\nUse context to answer: \n{context}\n\nAnswer with I don't know if you are not absolutely sure. \n\n"

In [69]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", new_system_prompt),
    fewshot_prompt, 
    ("user", "{user_prompt}")
])


In [70]:
fewshot_prompt.input_variables

[]

In [71]:
# First call without context to see if LLM knows 
pipeline_with_context = (
    {
        "user_prompt": lambda x : x["user_prompt"], 
        "context" : lambda x : x["context"]
    }
    | prompt_template
    | google_llm
)

answer = pipeline_with_context.invoke({"user_prompt" : user_prompt, "context": context})
answer.content[0]['text']

'The magnitude of the earthquake recorded in Taiwan on 27 Dec 2025 was **7.0** on the Richter Scale.'

# Chain of Thought Prompting 
More useful where we want LLM to think before responding to the question asked. 
Frontier models these day typically use chain of thought by default but for smaller LLMs this can be really useful.

In [101]:
# for this get the ollama 3.1 model 
from langchain_ollama import OllamaLLM

ollama_llm = OllamaLLM(temperature=0.0, model="gemma3:12b")

In [102]:
# no chain of thoughts system prompt 
no_cots_system_prompt = """
You are a helpful assistant and answer questions asked by the user. 

You MUST answer questions directly without any other text or explanation
"""

In [103]:
no_cots_prompt_template = ChatPromptTemplate([
    ("system", no_cots_system_prompt), 
    ("user", "{query}")
])

In [104]:
query = """ How many keystrokes are needed to type numbers between 1 to 500"""

no_cots_pipeline = no_cots_prompt_template | ollama_llm
no_cots_result = no_cots_pipeline.invoke({"query" : query})

In [105]:
no_cots_result

'1120'

``` 
the actual answer is 1392 

In [106]:
# lets use chain of thoughts 
cots_system_prompt = """
You are a helpful assistant and answer questions asked by the user. 
To answer the question, you must: 
- List systematically and precisely details of all subproblem that needs to be solved to answer the question 
- Solve each subproblem INDIVIDUALLY and in sequence 
- Finally, use everything you have worked through to provide the final answer
"""

In [107]:
cots_prompt_template = ChatPromptTemplate([
    ("system", cots_system_prompt), 
    ("user", "{query}")
])

In [108]:
cots_pipeline = cots_prompt_template | ollama_llm

In [109]:
cots_result = cots_pipeline.invoke({"query" : query})

In [110]:
import IPython
IPython.display.Markdown(cots_result)

Okay, let's break this down systematically. We need to calculate the total number of keystrokes required to type the numbers from 1 to 500.

**Subproblem 1: Keystrokes for 1-digit numbers (1-9)**

*   Each 1-digit number requires 1 keystroke.
*   There are 9 such numbers (1 to 9).
*   Total keystrokes: 9 * 1 = 9

**Subproblem 2: Keystrokes for 2-digit numbers (10-99)**

*   Each 2-digit number requires 2 keystrokes.
*   There are 99 - 10 + 1 = 90 such numbers.
*   Total keystrokes: 90 * 2 = 180

**Subproblem 3: Keystrokes for 3-digit numbers (100-499)**

*   Each 3-digit number requires 3 keystrokes.
*   There are 499 - 100 + 1 = 400 such numbers.
*   Total keystrokes: 400 * 3 = 1200

**Subproblem 4: Keystrokes for the number 500**

*   The number 500 requires 3 keystrokes.

**Subproblem 5: Total Keystrokes**

*   Add the keystrokes from all the subproblems.

**Solving the Subproblems:**

*   **Subproblem 1:** 9 keystrokes
*   **Subproblem 2:** 180 keystrokes
*   **Subproblem 3:** 1200 keystrokes
*   **Subproblem 4:** 3 keystrokes

**Final Answer:**

Total keystrokes = 9 + 180 + 1200 + 3 = 1392

Therefore, 1392 keystrokes are needed to type the numbers from 1 to 500.